In [2]:
pip install "torch>=2.2" "transformers>=4.45" pillow faiss-cpu numpy


  Using cached pillow-11.3.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 14.6 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 14.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 6.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.2 MB/s  0:00:00
Using cached pillow-11.3.0-cp311-cp311-macosx_11_0_arm64.whl (4.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.2 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 9.2 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.2 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 9.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import numpy as np, torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModel

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID = "facebook/dinov2-base"  # open model

processor = AutoImageProcessor.from_pretrained(MODEL_ID)
model = AutoModel.from_pretrained(MODEL_ID).to(DEVICE).eval()

@torch.inference_mode()
def embed_image(path: str) -> np.ndarray:
    img = Image.open(path).convert("RGB")
    inputs = processor(images=img, return_tensors="pt").to(DEVICE)
    out = model(**inputs)
    vec = torch.nn.functional.normalize(out.pooler_output.squeeze(0), dim=0)
    return vec.cpu().numpy()


/Users/nathanmarraccini/Desktop/Python/Anomaly Detection/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
ref_paths = [
    "images/7up_bottle.jpg",
    "images/coke_zero.jpg", 
    "images/coke.jpg",
    "images/pepsi_bottle.jpg",
    "images/pepsi.jpg",
    "images/sprite.jpg"
]

ref_vecs = np.stack([embed_image(p) for p in ref_paths], axis=0)
np.savez("dinov2_refs.npz", paths=np.array(ref_paths), vecs=ref_vecs)
print("Saved 10 embeddings → dinov2_refs.npz")


Saved 10 embeddings → dinov2_refs.npz


In [6]:

q = embed_image("images/pepsi_test.jpg")
sims = ref_vecs @ q            # both are L2-normalized → cosine similarity
best = int(np.argmax(sims))
top3_idx = np.argsort(-sims)[:3]
print("Most similar:", ref_paths[best], "score:", float(sims[best]))
print("Top-3:", [(ref_paths[i], float(sims[i])) for i in top3_idx])


Most similar: images/pepsi.jpg score: 0.832436203956604
Top-3: [('images/pepsi.jpg', 0.832436203956604), ('images/coke_zero.jpg', 0.7735756635665894), ('images/coke.jpg', 0.7493051886558533)]
